In [12]:
from pypdf import PdfReader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain_chroma import Chroma
from pprint import pprint
from langchain_groq import ChatGroq
import os
# from langchain_community.llms import HuggingFaceEndpoint
from langchain_community.llms import HuggingFaceTextGenInference
from langchain_huggingface import ChatHuggingFace
# import huggingface_hub
# from huggingface_hub import whoami

In [11]:
# Read the API key from a file
with open("C:\Programming\doc reader chatbot\groq2.txt", "r") as file:
    apiKey = file.read().strip()
# Set the API key as an environment variable
os.environ["GROQ_API_KEY"] = apiKey

<>:2: SyntaxWarning: invalid escape sequence '\P'
<>:2: SyntaxWarning: invalid escape sequence '\P'
C:\Users\mmibr\AppData\Local\Temp\ipykernel_35032\3315571182.py:2: SyntaxWarning: invalid escape sequence '\P'
  with open("C:\Programming\doc reader chatbot\groq2.txt", "r") as file:


In [13]:
def processText(text):
    # Split the text into chunks using Langchain's CharacterTextSplitter
    textSplitter = CharacterTextSplitter(
        # separator="\n",
        chunk_size = 1000,
        chunk_overlap = 50,
        # length_function=len
    )
    chunks = textSplitter.split_text(text)
    print(f"Split {len(text)} documents into {len(chunks)} chunks.")

    # Convert the chunks of text into embeddings to form a knowledge base
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
    db = Chroma.from_texts(chunks, embeddings)

    return db

In [18]:
# TODO: Understand this function fully
def analyzeResults(query, results):
    prompt_template = """
    Answer the query based only on the given context:
    {context}
    ---
    Answer the query based on the above context: {query}
    """

    context = "\n\n---\n\n".join([doc.page_content for doc, _score in results])

    prompt = prompt_template.format(context = context, query = query)

    llm = ChatGroq()
    # llm = ChatGroq(model="llama3-8b-8192")
    # llm = ChatGroq(model = "llama-3.1-70b-versatile")
    # response = llm.invoke(prompt)

    return response

In [19]:
file = "the-inspector-general--2.pdf"
pdf = PdfReader(file)
print(len(pdf.pages))

80


In [20]:
text = ""
for page in pdf.pages:
    text += page.extract_text()
print(len(text))

151431


In [21]:
db = processText(text)


Split 151431 documents into 1 chunks.


c:\Users\mmibr\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [22]:
query = "What is the premise of the story?"

In [23]:
results = db.similarity_search_with_score(query)
pprint(results)

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


[(Document(page_content='The\tInspector-General\n\t\nBy\n\t\nNikolái\tVasí\nlievi\nch\tGó\ngol\n\t\n\t\n\t\n\t\t\nCHARACTERS\tOF\tTHE\tPLAY\n\t\nANTON\tANTONOVICH\tSKVOZNIK-DMUKHANOVSKY,\tthe\n\t\nGovernor.\nANNA\tANDREYEVNA,\this\twife.\nMARYA\tANTONOVNA,\this\tdaughter.\nLUKA\tLUKICH\tKHLOPOV,\tthe\tInspector\tof\tSchools.\nHis\tWife.\nAMMOS\tFIODOROVICH\tLIAPKIN-TIAPKIN,\tthe\tJudge.\nARTEMY\tFILIPPOVICH\tZEMLIANIKA,\tthe\tSuperintendent\tof\n\t\nCharities.\nIVAN\tKUZMICH\tSHPEKIN,\tthe\tPostmaster.\nPIOTR\tIVANOVICH\tDOBCHINSKY.\t}\nPIOTR\tIVANOVICH\tBOBCHINSKY.\t}\tCountry\tSquires.\nIVAN\tALEKSANDROVICH\tKHLESTAKOV,\tan\tofficial\tfrom\tSt.\n\t\nPetersburg.\nOSIP,\this\tservant.\nCHRISTIAN\tIVANOVICH\tHÜBNER,\tthe\tdistrict\tDoctor.\nFIODR\tANDREYEVICH\tLULIUKOV.\tex-officials,\nIVAN\tLAZAREVICH\tRASTAKOVSKY.\tpersonages\nSTEPAN\tIVANOVICH\tKOROBKIN.\tof\tthe\ttown.\nSTEPAN\tILYICH\tUKHOVERTOV,\tthe\tPolice\tCaptain.\nSVISTUNOV\n.\nPUGOVITZYN.\n\t\nPolice\tSergeants.\nDERZHIMORDA

In [24]:
response = analyzeResults(query, results)

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `mixtral-8x7b-32768` in organization `org_01j589x79ae9m9gdyq6bq3qk7p` on tokens per minute (TPM): Limit 5000, Used 0, Requested 76069. Please try again in 14m12.828s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}